# Regression model

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from relife.models import ProportionalHazard, Weibull, Gompertz
from relife.datasets import load_insulator_string

ModuleNotFoundError: No module named 'relife.models'

In [ ]:
cp = 1
cf = 5
discounting_rate = 0.05

time, event, entry, *covar = load_insulator_string()
print(time)
print(covar)
covar = np.column_stack(covar)

In [ ]:
distrib = Weibull(5, 0.03)

In [ ]:
ProportionalHazard(distrib, (1, 2, 3)).hf(2, np.random.random((10, 3)))

In [ ]:
ph = ProportionalHazard(Gompertz()).fit(time, covar, event=event, entry=entry)

In [ ]:
# plot hazard function for some individuals
i, j, k = 12, 1073, 7999
t = np.linspace(0, 200, num=1000)
plt.plot(t, ph.hf(t, covar)[i], label="Individu i")
plt.plot(t, ph.hf(t, covar)[j], label="Individu j")
plt.plot(t, ph.hf(t, covar)[k], label="Individu k")
plt.xlabel("Time")
plt.ylabel("Fonction de défaillance")
plt.legend()
plt.show()

In [ ]:
# plot survival functions of some individuals
plt.plot(t, ph.sf(t, covar)[i], label="Individu i")
plt.plot(t, ph.sf(t, covar)[j], label="Individu j")
plt.plot(t, ph.sf(t, covar)[k], label="Individu k")
plt.xlabel("Time")
plt.ylabel("Fonction de survie")
plt.legend()
plt.show()

In [ ]:
distri = ph.get_distribution(covar[[i, j, k]])

In [ ]:
plt.plot(t, distri.sf(t)[0], label="Individu i")
plt.plot(t, distri.sf(t)[1], label="Individu j")
plt.plot(t, distri.sf(t)[2], label="Individu k")

In [ ]:
from relife.models import LeftTruncatedModel

left_truncated_distri = LeftTruncatedModel(distri)

In [ ]:
a0 = np.array([30, 30, 30]).reshape(-1,1)
plt.plot(t, left_truncated_distri.sf(t, a0)[0], label="Individu i")
plt.plot(t, left_truncated_distri.sf(t, a0)[1], label="Individu j")
plt.plot(t, left_truncated_distri.sf(t, a0)[2], label="Individu k")

In [ ]:
from relife.models import AgeReplacementModel

ar_distri = AgeReplacementModel(distri)

In [ ]:
ar = np.array([75, 75, 75]).reshape(-1,1)
plt.plot(t, ar_distri.sf(t, ar)[0], label="Individu i")
plt.plot(t, ar_distri.sf(t, ar)[1], label="Individu j")
plt.plot(t, ar_distri.sf(t, ar)[2], label="Individu k")

## Politique de remplacement par âge dans un modèle à covariables

Il est possible de fitter une politique de remplacement par âge d'après un modèle proportionnel. Il suffit de passer les covariables dans l'argument `model_args` de la classe AgeReplacementPolicy.

In [ ]:
# Calculons la politique et l'âge de remplacement pour les individus i, j, k définis ci-dessus:
policy_ph = AgeReplacementPolicy(
    ph, cf, cp, discounting_rate=discounting_rate, model_args=(covar[[i, j, k]],), nb_assets=3
).fit()
print(policy_ph.ar)